In [ ]:
import itertools

import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from numba import njit
from tqdm import tqdm

from Utils import extract_symbol_timeframe

# === Чтение CSV ===
path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)
df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
symbol, timeframe = extract_symbol_timeframe(path)

# === Преобразование в numpy float32 ===
highs = df["high"].to_numpy(dtype=np.float32)
lows = df["low"].to_numpy(dtype=np.float32)
closes = df["close"].to_numpy(dtype=np.float32)
times = df["open_time"].values.astype("datetime64[ns]")

# === Параметры сетки ===

impulse_range = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25], dtype=np.float32)
tp_range = np.array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 8, 9, 10], dtype=np.float32)
window_range = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 20, 25, 30, 35, 40, 50, 60, 70], dtype=np.int32)
stop_loss_pct = np.float32(99.0)

combos = list(itertools.product(impulse_range, tp_range, window_range))

# === Ядро стратегии ===
@njit
def simulate_strategy_core(highs, lows, closes, impulse_mask, tp, window, stop_loss_pct):
    n = closes.shape[0]
    balance = 10.0
    in_trade = False
    durations = []

    i = 0
    while i < n:
        if not in_trade and impulse_mask[i]:
            entry_price = closes[i]
            take_profit = entry_price * (1 + tp / 100)
            stop_loss = entry_price * (1 - stop_loss_pct / 100)
            in_trade = True

            j = i + 1
            while j < n:
                if highs[j] >= take_profit or lows[j] <= stop_loss:
                    break
                j += 1
            if j >= n:
                break

            if highs[j] >= take_profit:
                exit_price = take_profit
            else:
                exit_price = stop_loss

            profit_pct = (exit_price / entry_price - 1) * 100
            balance *= (1 + profit_pct / 100)
            durations.append(j - i)
            in_trade = False
            i = j
        else:
            i += 1

    total_pct = (balance / 10.0 - 1) * 100
    return total_pct, np.array(durations, dtype=np.float32)

# === Функция обработки одной комбинации ===
def process_combo(combo):
    impulse_threshold, tp, window = combo

    rolling_max = pd.Series(closes).rolling(window=window).max().to_numpy(dtype=np.float32)
    rolling_min = pd.Series(closes).rolling(window=window).min().to_numpy(dtype=np.float32)
    drawdowns = (rolling_min / rolling_max - 1) * 100
    impulse_mask = drawdowns <= -impulse_threshold

    total_pct, durations = simulate_strategy_core(highs, lows, closes, impulse_mask, tp, window, stop_loss_pct)

    if durations.size > 0:
        d_min = np.min(durations)
        d_max = np.max(durations)
        d_mean = np.mean(durations)
        d_median = np.median(durations)
        last_entry_index = np.where(impulse_mask)[0][-1]
        last_entry_time = times[last_entry_index]
    else:
        d_min = d_max = d_mean = d_median = 0.0
        last_entry_time = pd.NaT

    return {
        "impulse_threshold": impulse_threshold,
        "take_profit": tp,
        "window": window,
        "profit_total_pct": total_pct,
        "duration_min": d_min,
        "duration_max": d_max,
        "duration_mean": d_mean,
        "duration_median": d_median,
        "last_entry_time": last_entry_time
    }

    """ Важно время в датафрейме будет указано 
    в единицах измерении как таймфрем датафрейма,
    то есть есть duration_min = 1, а таймфейм 1час, 
    то минимальное время сделки будет равно 1часу. 
    Это чуть не точно, потому что сделка могла закрыться 
    и за 10 минут в районе одной свечи, но мы округляем до тайм фрейма, 
    по причине - Numba не умеет работать с массивами datetime64 как с обычными числами.
    """

# === Параллельная обработка ===
results = Parallel(n_jobs=-1)(
    delayed(process_combo)(combo) for combo in tqdm(combos, desc="⚙️ Grid Search")
)

# === Итоговая таблица ===
grid_df = pd.DataFrame(results)
grid_df = grid_df.sort_values("profit_total_pct", ascending=False)

# === Сохранение ===

indicator_name = "simple_strategy"
filename = f"{symbol}_{timeframe}_{indicator_name}_{len(combos)}_combos.csv"
grid_df.to_csv(filename, index=False)

print(f"📁 Результаты сохранены в: {filename}")
print(f"🏆 Топ-5 лучших стратегий для {symbol} на таймфрейме {timeframe}:")
grid_df.head(5)
